In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install --upgrade imbalanced-learn

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
from PIL import ImageFile

# Allow loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ================================
# Dataset loader
# ================================
def load_dataset(directory, target_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    class_mapping = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        if not os.path.isdir(class_dir):
            continue

        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    img_path = os.path.join(class_dir, filename)
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Skipping corrupted image: {img_path}")
                        continue
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, target_size)
                    img = img.astype('float32') / 255.0
                    images.append(img)
                    labels.append(class_mapping[class_name])
                except Exception as e:
                    print(f"Error loading {img_path}: {str(e)}")
                    continue

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, {v: k for k, v in class_mapping.items()}

# ================================
# Load datasets
# ================================
print("Loading training data...")
train_images, train_labels, inv_class_mapping = load_dataset(
    "/kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/train"
)

print("Loading test data...")
test_images, test_labels, _ = load_dataset(
    "/kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/val"
)

num_classes = len(inv_class_mapping)

# One-hot encode labels
y_train = keras.utils.to_categorical(train_labels, num_classes=num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes=num_classes)

# ================================
# Build Model
# ================================
def create_model():
    base_model = keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze feature extractor

    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

model = create_model()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ================================
# Train model
# ================================
print("\nTraining model...")
history = model.fit(
    train_images, y_train,
    epochs=10,
    batch_size=16,
    validation_data=(test_images, y_test),
    verbose=1
)

# ================================
# Save model
# ================================
model.save('maize_pest_mobilenet.keras')
print("\nModel saved successfully!")


2025-07-20 07:16:27.350106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752995787.529226      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752995787.581324      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading training data...
Skipping corrupted image: /kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/train/leaf beetle/leaf beetle208_.jpg
Skipping corrupted image: /kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/train/leaf beetle/leaf beetle458_.jpg


Corrupt JPEG data: 411 extraneous bytes before marker 0xc0
Corrupt JPEG data: 8635 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


Skipping corrupted image: /kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/train/leaf beetle/leaf beetle326_.jpg
Skipping corrupted image: /kaggle/input/detail/Maize-Pest-Detection-And-Identification-master/DataSet on Maize Pests/Pest Detection/train/leaf beetle/leaf beetle573_.jpg


Corrupt JPEG data: 60 extraneous bytes before marker 0xdb
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1115 extraneous bytes before marker 0xb4


...

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training model...
Epoch 1/10


I0000 00:00:1752995903.900779     100 service.cc:148] XLA service 0x7abe84004300 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752995903.901598     100 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752995904.771244     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


  9/407 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.1883 - loss: 2.8964       

I0000 00:00:1752995908.409168     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


407/407 ━━━━━━━━━━━━━━━━━━━━ 31s 49ms/step - accuracy: 0.6454 - loss: 1.1991 - val_accuracy: 0.8841 - val_loss: 0.3755
Epoch 2/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.8789 - loss: 0.3928 - val_accuracy: 0.8879 - val_loss: 0.3657
Epoch 3/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9078 - loss: 0.2986 - val_accuracy: 0.9026 - val_loss: 0.3086
Epoch 4/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9177 - loss: 0.2384 - val_accuracy: 0.9080 - val_loss: 0.3052
Epoch 5/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9389 - loss: 0.1931 - val_accuracy: 0.9015 - val_loss: 0.3623
Epoch 6/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9445 - loss: 0.1602 - val_accuracy: 0.9020 - val_loss: 0.3397
Epoch 7/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9481 - loss: 0.1430 - val_accuracy: 0.9129 - val_loss: 0.2942
Epoch 8/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9582 - loss: 0.1206 - val_accuracy: 0.91

In [3]:
print(inv_class_mapping)

{0: 'ants', 1: 'aphids', 2: 'catterpillar', 3: 'corn worm', 4: 'earwig', 5: 'fall armyworm', 6: 'grasshoper', 7: 'leaf beetle', 8: 'mole cricket', 9: 'red spider', 10: 'slug', 11: 'snail', 12: 'stem borrer', 13: 'weevil'}
